<a href="https://colab.research.google.com/github/alexedett/IA/blob/main/clasificar_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
#sklearn es una libreria de aprendizaje automatico, en el cual vamos a tener los algoritmos, se va a usar el de clasificacion
#pandas para lectura del archivo como data frame
import pandas as pd
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from google.colab import files
uploaded = files.upload()



#Leemos el dataset
spam_or_ham = pd.read_csv("spam.csv", encoding="latin-1")[["v1", "v2"]]
spam_or_ham.columns = ["label", "text"] #Nombre de las columnas
spam_or_ham.head() #descomentar para ver si cargo bien el data set

#----------------------------------------------#

#miramos cuantos spam y ham hay en el data set por medio de la columna label
#spam_or_ham["label"].value_counts()

#----------------------------------------------#
#Metodo de optimizacion para eliminar muestras innecesarias

punctuation = set(string.punctuation)
 #Contiene los signos de puntuacion estandar

#Devuelve una lista de tokens sin signos de puntuacion y en minuscula
def tokenize(sentence):
  tokens = []
  for token in sentence.split(): #Divide las oraciones en palabras (tokens preliminares) usando los espacios
    new_token = []
    for character in token: #Recorre cada letra de la palabra actual
      if character not in punctuation: #Verificar si la letra es un signo, si no se guarda en minuscula
        new_token.append(character.lower())
    if new_token: #si hay caracteres en la lista los unira para hacer una palabra con todo en minuscula y sin caracteres especiales
      tokens.append("".join(new_token))
  return tokens

#spam_or_ham.head()["text"].apply(tokenize) #A la culumna de texto del data set le aplicamos el algoritmo, a la cabeza del data set

#----------------------------------------------#

#Convertimos en una matriz de conteo de palabras
demo_vectorizer = CountVectorizer(
    tokenizer= tokenize, #le pasamos la funcion que habiamos creado para filtrar datos
    binary= True #Dice si la palabra aparece o no
)

#----------------------------------------------#

#Dividimos los datos en parte de prueba y parte de entrenamiento
train_text, test_text, train_labels, test_labels = train_test_split(spam_or_ham["text"], spam_or_ham["label"], stratify=spam_or_ham["label"])


#print(f"Training examples: {len(train_text)}, texting examples {len(test_text)}") #mostrar la cantidad de datos entrenados y datos de prueba

#----------------------------------------------#

real_vectorizer = CountVectorizer(tokenizer=tokenize, binary=True) #Crea un vector no entrenado para contar las veces que sale una palabra
train_x = real_vectorizer.fit_transform(train_text) #Aprende las palabras unicas y las convierte en un vector binario con ese vocabulario
test_x = real_vectorizer.transform(test_text) #Transforma el conjunto de prueba en vectores

#----------------------------------------------#

classifier = LinearSVC() #Crea un calsificador de maquinas de vectores
classifier.fit(train_x, train_labels) #entrena el modelo con los parametros
LinearSVC(C= 1.0,
          class_weight =None,
          dual = True, fit_intercept=True,
          intercept_scaling=1,
          loss = "squared_hinge",
          max_iter =1000,
          multi_class = "ovr",
          penalty="l2",
          random_state=None,
          tol= 0.0001,
          verbose = 0) #Otra instancia con todos los parametros explicitos

#----------------------------------------------#

predicciones = classifier.predict(test_x) #Aplica el modelo ya entrenado para clasificar spam y ham
accuracy = accuracy_score(test_labels, predicciones) #Calcula el porcentaje de aciertos con la etiqueta real
print(f"Accuracy: {accuracy:.4%}") #Imprime el porcentaje de aciertos


#----------------------------------------------#
#Las nuevas frases que se quieren clasificar
frases = [
    "Are you looking to redesign your webside with new modern look and feel?",
    "Please. Send me a confirmation of complete and permanent erasure of the personal data",
    "You have been selected to win a FREE suscription to our service",
    "We're contacting you because the webhook endpoint associated with your account in test mode has been failing",
    "Confirma tu cuenta de Facebook con el siguiente link",
    "You have been selected to participate in a free servise",
    "Congratulations! You've won a $1,000 gift card. Click here to claim your prize.",
    "Urgent: Your account has been compromised. Verify your details immediately.",
    "You have been selected for a FREE iPhone! Just pay shipping.",
    "Claim your lottery winnings now before they expire!",
    "Earn $500 per day working from home. No experience needed.",
    "Limited-time offer! Get 90 off on all products today only.",
    "Your PayPal account has been suspended. Log in to resolve the issue.",
    "This is not a scam! You've been chosen for an exclusive cash prize.",
    "Access adult content for free – no credit card required!",
    "Get your miracle weight-loss pills now, no diet or exercise needed!"
]

#----------------------------------------------#

frases_x = real_vectorizer.transform(frases) #Convierte las frases en vectores binarios
predicciones = classifier.predict(frases_x) #Pasa los vectores al modelo entrenado y predice si es spam o no

#----------------------------------------------#

#Muestra los resultados de la clasificacion de cada frase

for text, label in zip(frases, predicciones):
  print(f"{label:5} - {text}")
  import imaplib
import email
from email.header import decode_header

# Conexión IMAP a Gmail
usuario = "lynnalejandra123@gmail.com"
contraseña = "jejt jemq ldyl hcwr"  # Usa una contraseña de aplicación, no la normal

mail = imaplib.IMAP4_SSL("imap.gmail.com")
mail.login(usuario, contraseña)
mail.select("inbox")

# Buscar correos no leídos
status, mensajes = mail.search(None, 'UNSEEN')
ids = mensajes[0].split()

for i in ids[:10]:  # Solo los primeros 10 no leídos
    status, datos = mail.fetch(i, "(RFC822)")
    for respuesta in datos:
        if isinstance(respuesta, tuple):
            mensaje = email.message_from_bytes(respuesta[1])
            asunto, cod = decode_header(mensaje["Subject"])[0]
            if isinstance(asunto, bytes):
                asunto = asunto.decode(cod or "utf-8")
            cuerpo = ""
            if mensaje.is_multipart():
                for parte in mensaje.walk():
                    tipo = parte.get_content_type()
                    if tipo == "text/plain":
                        cuerpo = parte.get_payload(decode=True).decode(errors="ignore")
                        break
            else:
                cuerpo = mensaje.get_payload(decode=True).decode(errors="ignore")

            # Clasificación
            texto = asunto + " " + cuerpo
            texto_vectorizado = real_vectorizer.transform([texto])
            pred = classifier.predict(texto_vectorizado)[0]
            print(f"[{pred.upper()}] {asunto}")



Saving spam.csv to spam (5).csv
Accuracy: 98.3489%
spam  - Are you looking to redesign your webside with new modern look and feel?
ham   - Please. Send me a confirmation of complete and permanent erasure of the personal data
spam  - You have been selected to win a FREE suscription to our service
ham   - We're contacting you because the webhook endpoint associated with your account in test mode has been failing
ham   - Confirma tu cuenta de Facebook con el siguiente link
ham   - You have been selected to participate in a free servise
spam  - Congratulations! You've won a $1,000 gift card. Click here to claim your prize.
ham   - Urgent: Your account has been compromised. Verify your details immediately.
ham   - You have been selected for a FREE iPhone! Just pay shipping.
ham   - Claim your lottery winnings now before they expire!
ham   - Earn $500 per day working from home. No experience needed.
ham   - Limited-time offer! Get 90 off on all products today only.
ham   - Your PayPal accoun

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


[HAM] 3 meses de Premium por $ 900,00: ¡Que comience la fiesta!
[HAM] ¡Quedan 7 días para votar virtualmente! 💻📱
[HAM] Complete la siguiente encuesta y participe en la
[SPAM] Agenda Cultural del 15 al 22 de febrero de 2019
[HAM] Tienes música nueva de Paulo Londra y de otros artistas que te encantan en tu Radar de Novedades.
[HAM] Ayúdanos a protegerte: Consejo de seguridad de Google
[SPAM] Agenda Cultural del 22 al 28 de febrero de 2019
[HAM] Música nueva de Sharlene y más de tus artistas favoritos: escúchalos ahora en tu Radar de Novedades
[HAM] Empieza a disfrutar hoy.
[HAM] Asiste a la 
